In [2]:
# preparation 
from math import e

def sigmoid(x):
    return 1/(1+e**x)

def sigmoid_prime(x):
    return sigmoid(x)*(1-sigmoid(x))

In [3]:
# Data set
import numpy as np

input_val = np.asarray([[0, 0],
                        [0, 1],
                        [1, 0],
                        [1, 1]])

target_or = np.asarray([[0],
                        [1],
                        [1],
                        [0]])

target_xor = np.asarray([[0],
                         [1],
                         [0],
                         [1]])

In [4]:
# Perceptron implementation
class Perceptron():
    def __init__(self, input_units:int, bias=True, act_func=sigmoid, alpha=0.01):
        
        # initialize weights
        if bias:
            self.weights = np.random.randn(input_units+1)
        else:
            self.weights = np.random.randn(input_units)
        
        # set learning rate
        self.alpha = alpha

        # set activation function
        self.act_func = act_func
        
        # allocating space for variables
        self.drive = int()
        self.activation = int()
        
    def fwd_step(self, inputs):
        self.inputs = np.insert(inputs, 0, 1)
        self.drive = self.weights @ self.inputs
        self.activation = self.act_func(self.drive)
        return self.activation
    
    def update(self, delta):
        self.weights -= self.alpha * (self.inputs * delta)

In [124]:
# MLP implementation
class MLP():
    def __init__(self, dim, act_func=sigmoid, act_func_prime=sigmoid_prime):
        self._dim = dim
        self.layers = [[Perceptron(self._dim[n], act_func=act_func) for _ in range(l)] for n, l in enumerate(self._dim[1:])]
        self.output = [0 for _ in range(self._dim[-1])]
        self.act_func_prime = sigmoid_prime
        
    def fwd_step(self, inputs):
        assert len(inputs) == self._dim[0]
        
        self.activations = [inputs]
        for layer in self.layers:
            activation = []
            for perceptron in layer:
                activation.append(perceptron.fwd_step(self.activations[-1]))
            self.activations.append(activation)
    
    def backprop(self, target):
        assert len(target) == self._dim[-1]
        
        self.errors = []
        # initial layer
        error = []
        for n, (target, output) in enumerate(zip(target, self.activations[-1])):
            error.append(-(target-output)*self.act_func_prime(self.layers[-1][n].drive))
        self.errors.append(error)
                
        for n, layer in enumerate(reversed(self.layers[1:])):
            # other layers
            layer_errors = np.zeros(len(layer[0].weights[1:]))
            for i, (perceptron, error) in enumerate(zip(layer, self.errors[-1])):
                layer_errors[i] = np.sum(error*np.asarray(perceptron.weights[1:]))*self.act_func_prime
            self.errors.append(layer_errors)
        
        
        for i, perceptron in enumerate(self.layers[-1]):
            perceptron.update(self.errors[0][i])

In [125]:
mlp = MLP([2, 4, 2, 1])
mlp.fwd_step([1, 1])
mlp.backprop([1])